In [ ]:
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np

image_count = 6*8
X = np.zeros((image_count, 92*112))
mean_face = np.zeros((92, 112))
for i in range(6):
    for j in range(8):
        image = cv.imread('images/s1/1.pgm', cv.IMREAD_GRAYSCALE)
        image_arr = np.array([pixel for row in image for pixel in row], dtype=np.float16)
        X[i*j] = image_arr


